El objetivo de este este código es implementar un MLP dada una partición, y dados los datos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [ ]:
!pip install comet_ml
from comet_ml import Experiment
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import joblib
import time
import matplotlib.pyplot as plt

sys.path.append('/content/drive/My Drive/repo_tesis/lib')
import classes as cl

In [ ]:
# Importo los datos
features = np.array(joblib.load("/content/drive/My Drive/repo_tesis/data/FEATURES_W200_I50_PCA95.joblib"))
label = np.array(joblib.load("/content/drive/My Drive/repo_tesis/data/label_W200_I50.joblib"))

%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"
# Importo particiones
indices_train = np.array(joblib.load('indices_train_W200_I50.pkl'))
indices_val = np.array(joblib.load('indices_val_W200_I50.pkl'))
indices_test = np.array(joblib.load('indices_test_W200_I50.pkl'))



/content/drive/My Drive/repo_tesis/archivos_generados_codigos


In [ ]:
start_time = time.time()

inputDim = features.shape[1]  # Esto lo cambié. Me fijo las dimensiones directamente en las dimensiones de la matriz
clasNum = np.max(label[:, 1]) + 1

# Borrar todo lo que no use de aqui para adelante en este punto

# Architecture 250 - 500 - 250
red = cl.Clasificador(model= "mlp", arch = [250,500,250], catNum = clasNum, featureNum = inputDim)
red.train(features[indices_train, :], label[indices_train, 1], features[indices_val, :], label[indices_val, 1])

end_time = time.time()
training_time = end_time - start_time
y_pred = red.predict(features[indices_test, :])

nombre = 'ronda1_exp_mlp_sep_rep_pca95_n1'

joblib.dump(red, nombre + '.pkl')
y_pred = red.predict(features[indices_test, :])
accuracy = accuracy_score(label[indices_test, 1], y_pred)
precision = precision_score(label[indices_test, 1], y_pred, average='weighted')
recall = recall_score(label[indices_test, 1], y_pred, average='weighted')
conf_matrix = confusion_matrix(label[indices_test, 1], y_pred)
# plt.savefig("confusion_matrix.png")

# Conectar con Comet
API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO' # Se puede encontrar en Settings(Arriba a la derecha en Comet)
exp = Experiment(api_key=API_KEY,
              project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
              auto_param_logging=False)
exp.set_name(nombre) # Nombre de este experimento
exp.add_tags(['exp1', 'mlp', 'sep_rep', 'pca95']) # Tags

exp.log_metric("accuracy", accuracy)
exp.log_metric("precision", precision)
exp.log_metric("recall", recall)
exp.log_metric("training_time", training_time)
exp.log_confusion_matrix(label[indices_test, 1], y_pred)
exp.log_parameter("ind_train", indices_train)
exp.log_parameter("ind_val", indices_val)
exp.log_parameter("ind_test", indices_test)
exp.log_text("Experimento separando los datos por repeticiones y usando MLP. En cuanto a las ventanas: W=200, I=50.")   # Comentario del experimento
exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
exp.end()

Epoch 1/50
2691/2691 [==============================] - 18s 6ms/step - loss: 1.2013 - accuracy: 0.5866 - val_loss: 1.3905 - val_accuracy: 0.5627
Epoch 2/50
2691/2691 [==============================] - 14s 5ms/step - loss: 0.7790 - accuracy: 0.7312 - val_loss: 1.4248 - val_accuracy: 0.5869
Epoch 3/50
2691/2691 [==============================] - 11s 4ms/step - loss: 0.5750 - accuracy: 0.8020 - val_loss: 1.6478 - val_accuracy: 0.5761
Epoch 4/50
2691/2691 [==============================] - 11s 4ms/step - loss: 0.4309 - accuracy: 0.8512 - val_loss: 1.8458 - val_accuracy: 0.5927
Epoch 5/50
2691/2691 [==============================] - 11s 4ms/step - loss: 0.3338 - accuracy: 0.8845 - val_loss: 2.1845 - val_accuracy: 0.5729
Epoch 6/50
2691/2691 [==============================] - 11s 4ms/step - loss: 0.2620 - accuracy: 0.9087 - val_loss: 2.3671 - val_accuracy: 0.5645
Epoch 7/50
2691/2691 [==============================] - 9s 3ms/step - loss: 0.2157 - accuracy: 0.9280 - val_loss: 2.5601 - val_acc

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/cab0e1da6f684ecdbfd0dd0ee3ffc97c

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/cab0e1da6f684ecdbfd0dd0ee3ffc97c
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.5662481259370314
COMET INFO:     precision     : 0.5815488540019711
COMET INFO:     recall        : 0.5662481259370314
COMET INFO:     training_time : 530.0921504497528
COMET INFO:   Others:
COMET INFO